## Writing a Simple Action Client
This tutorial covers using the `action_client` library to create a Fibonacci simple action client in Python.

In [ ]:
import rospy
import time
from __future__ import print_function
from actionlib import SimpleActionClient
from actionlib_tutorials.msg import FibonacciAction, FibonacciGoal

In [ ]:
rospy.init_node('fibonacci_client_py')

In [ ]:
client = SimpleActionClient('fibonacci', FibonacciAction)
client.wait_for_server()

In [ ]:
goal = FibonacciGoal(order=5)
client.send_goal(goal)

In [ ]:
client.wait_for_result()
result = client.get_result()
print('[Result]    State: %d' % client.get_state())
print('[Result]   Status: %s' % client.get_goal_status_text())
print('[Result] Sequence:', result.sequence)

## Processing feedback

In [ ]:
def feedback_cb(feedback):
    print("[Feedback] Sequence:", feedback.sequence)

In [ ]:
goal = FibonacciGoal(order=5)
client.send_goal(goal, feedback_cb=feedback_cb)

In [ ]:
client.wait_for_result()
result = client.get_result()
print('[Result]    State: %d' % client.get_state())
print('[Result]   Status: %s' % client.get_goal_status_text())
print('[Result] Sequence:', result.sequence)

## Cancelling a request

In [ ]:
goal = FibonacciGoal(order=5)
client.send_goal(goal)
time.sleep(1.5)
client.cancel_goal()

In [ ]:
client.wait_for_result()
result = client.get_result()
print('[Result]    State: %d' % client.get_state())
print('[Result]   Status: %s' % client.get_goal_status_text())
print('[Result] Sequence:', result.sequence)

## Preempting a previous goal

The `SimpleActionServer` implements a single goal policy: accepting a new goal implies successful preemption of any old goal and the status of the old goal will be changed automatically to reflect this.

In [ ]:
goal = FibonacciGoal(order=5)
client.send_goal(goal)
time.sleep(1.5)
new_goal = FibonacciGoal(order=8)
client.send_goal(new_goal)

In [ ]:
client.wait_for_result()
result = client.get_result()
print('[Result]    State: %d' % client.get_state())
print('[Result]   Status: %s' % client.get_goal_status_text())
print('[Result] Sequence:', result.sequence)

## Action aborted by server

In [ ]:
goal = FibonacciGoal(order=105)
client.send_goal(goal)

In [ ]:
client.wait_for_result()
result = client.get_result()
print('[Result]    State: %d' % client.get_state())
print('[Result]   Status: %s' % client.get_goal_status_text())
print('[Result] Sequence:', result.sequence)

For going back to the main page, please close the other tabs and click on the following link:
[Go back to the main page](../../README.ipynb)